In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib
matplotlib.rcParams.update({'font.size': 12,'font.family':'serif','font.serif':['Computer Modern'],"text.usetex" : True,})

import ipywidgets as widgets
import datetime
import pandas as pd
from tqdm.auto import tqdm
import os
import nmrglue
import hypermri
import hypermri.utils.utils_anatomical as ut_anat
import hypermri.utils.utils_spectroscopy as ut_spec
import hypermri.utils.utils_fitting as ut_fitting
from scipy.optimize import curve_fit
from matplotlib import cm
from astropy.modeling import models, fitting

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.dates as mdates
    
import sys
# define paths:
sys.path.append('../../')
import TEMPLATE
# get paths:
repopath, basepath, savepath = TEMPLATE.import_all_packages()
savepath=savepath+'AnimalSliceSpecResults/'
from mpl_interactions import image_segmenter_overlayed


# Autoreload extension so that you dont have to reload the kernel every time something is changed in the hypermri or magritek folders
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [ ]:
single_column_width = 6.9 # inches
from hypermri.utils.utils_spectroscopy import find_npeaks as ut_find_npeaks
from hypermri.utils.utils_fitting import temperature_from_frequency

In [ ]:
scans = hypermri.BrukerDir(basepath+'/')
temp_df = hypermri.utils.utils_general.load_pc_sam_temp_file(basepath+'AnimalTemperatures/.txt','',(2*3600)+8,False)

In [ ]:
nspect_38deg = scans[11]
nspect_31deg = scans[30]
coronal = scans[28]
t2w_like_nspect = scans[18]

# Fitting spectra and extracting time curves for 38°C experiment

In [ ]:
hyper = nspect_31deg

print(datetime.datetime.strptime(hyper.acqp['ACQ_time'][1:11]+'/'+hyper.acqp['ACQ_time'][12:20],
                                                            '%Y-%m-%d/%H:%M:%S'))

fig,ax=plt.subplots(1)
@widgets.interact(rep=(0,350-1,1))
def update(rep=0):
    ax.cla()
    ax.plot(hyper.ppm,hyper.spec[rep,:])

In [ ]:
sig_38=range(13,50)
sig_31=range(13,52)

## Load previously fitted spectra

In [ ]:
filename = '_fit_spectra.pkl'

fit_results = hypermri.utils.utils_general.load_as_pkl(dir_path=savepath,
            filename=filename)
fit_spectrums_38 = fit_results["fit_spectrums_38"]
fit_spectrums_31 = fit_results["fit_spectrums_31"]
fit_freqs_38 = fit_results["fit_freqs_38"]
fit_freqs_31 = fit_results["fit_freqs_31"]
fit_amps_38 = fit_results["fit_amps_38"]
fit_amps_31 = fit_results["fit_amps_31"]
fit_t2s_38 = fit_results["fit_t2s_38"]
fit_t2s_31 = fit_results["fit_t2s_31"]
fit_stds_38 = fit_results["fit_stds_38"]
fit_stds_31 = fit_results["fit_stds_31"]

fit_params_31 = fit_results["fit_params_31"]
fit_params_38 = fit_results["fit_params_38"]
fit_freqs_ppm_38 = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs_38), hz_axis=fit_params_38["freq_range_Hz"], ppm_axis=fit_params_38["freq_range_ppm"], ppm_axis_flipped=False)
fit_freqs_ppm_31 = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs_31), hz_axis=fit_params_31["freq_range_Hz"], ppm_axis=fit_params_31["freq_range_ppm"], ppm_axis_flipped=False)

## frequency errors
fit_freqs_diff_stds_38 = np.sqrt(np.abs(fit_stds_38[0,0,0,0,:,0,0,1]**2) +
                                        np.abs(fit_stds_38[0,0,0,0,:,0,1,1]**2))
## frequency errors
fit_freqs_diff_stds_31 = np.sqrt(np.abs(fit_stds_31[0,0,0,0,:,0,0,1]**2) +
                                        np.abs(fit_stds_31[0,0,0,0,:,0,1,1]**2))
print('Loaded ',savepath+'/'+filename)


# Fitting spectra and extracting time curves for 38°C experiment

In [ ]:
experiment=nspect_38deg
repetitions_to_fit=sig_38
metabs = ['pyruvate', 'lactate', 'alanine', 'pyruvatehydrate', 'urea']

fit_params = {}

fit_params["zoomfactor"] = 1.5
fit_params["max_t2_s"] = 0.1
fit_params["min_t2_s"] = 0.001
fit_params["range_t2s_s"] = 0.05

# get the indices of the peaks:
fit_params["metabs"] = metabs
fit_params["fit_range_repetitions"] = repetitions_to_fit
fit_params["range_freqs_Hz"] = 25
fit_params["cut_off"] = 70
fit_params["niter"] = 1 # number of iterations:
fit_params["npoints"] = 21 # number of tested points per iteration:
fit_params["rep_fitting"] = 11 # number of tested points per iteration:
fit_params["provided_dims"] = ["reps", "fid"]
fit_params["use_all_cores"] = True

fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)


# get the indices of the peaks:
# get the indices of the peaks:
peak_indices = ut_spec.find_npeaks(input_data=np.real(np.sum(np.fft.fftshift(np.fft.fft(np.fft.ifft(np.fft.ifftshift(experiment.complex_spec))[:,fit_params["cut_off"]:])),axis=0)),
                              freq_range=fit_params['freq_range_ppm'],
                              npeaks=len(metabs),
                              find_peaks_params={'distance':150},
                                   plot=True)
# diff literature and measured:


fit_params["metabs_freqs_ppm"] = fit_params["freq_range_ppm"][peak_indices]
    #overwrite the peak values since the hydrate peak is not found for some reason
#fit_params["metabs_freqs_ppm"] = [163.6,171.1,175.9,179.9,183.3]
#fit_params["metabs_freqs_Hz"] = [None, None, None,None]

fit_params["niter"] = 1 # number of iterations:
fit_params["npoints"] = 21 # number of tested points per iteration:
fit_params["rep_fitting"] = 11 # number of tested points per iteration:

fit_params["provided_dims"] = ["reps", "fid"]
fit_params["use_all_cores"] = True

fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)
fit_params_38=fit_params
fit_spectrums_38, fit_amps_38, fit_freqs_38, fit_t2s_38, fit_stds_38  = ut_fitting.fit_data_pseudo_inv(input_data=experiment.complex_spec,
                                                                             data_obj=experiment,
                                                                  fit_params=fit_params_38,
                                                              use_multiprocessing=True)

fit_freqs_ppm_38 = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs_38), hz_axis=fit_params_38["freq_range_Hz"], ppm_axis=fit_params_38["freq_range_ppm"], ppm_axis_flipped=False)


In [ ]:
## frequency errors
fit_freqs_diff_stds_38 = np.sqrt(np.abs(fit_stds_38[0,0,0,0,:,0,0,1]**2) +
                                        np.abs(fit_stds_38[0,0,0,0,:,0,1,1]**2))

In [ ]:
def plot_fit_results(fit_spectrums,fit_freqs,signal_range,experiment,fit_params):
    ppm=ut_spec.get_freq_axis(experiment,cut_off=fit_params['cut_off'])
    fig,ax=plt.subplots(1)      
    
    @widgets.interact(rep = (signal_range[0],signal_range[-1],1))
    def update(rep=signal_range[0]):
        ax.cla()
        ax.plot(ppm,np.real(np.sum(np.squeeze(fit_spectrums)[:,rep,:],axis=1)))
        ax.plot(ppm,np.real(np.fft.fftshift(np.fft.fft(experiment.fids[rep,:][fit_params["cut_off"]::]))),c='k',alpha=0.3)
        ax.set_title((np.squeeze(fit_freqs)[rep,0])-np.squeeze(fit_freqs)[rep,1])


In [ ]:
plot_fit_results(fit_spectrums_38,fit_freqs_38,sig_38,nspect_38deg,fit_params_38)

# Fitting spectra for the 31 degree measurement

In [ ]:
experiment=nspect_31deg
repetitions_to_fit=sig_31

metabs = ['pyruvate', 'lactate', 'alanine', 'pyruvatehydrate', 'urea']

fit_params = {}

fit_params["zoomfactor"] = 1.5
fit_params["max_t2_s"] = 0.1
fit_params["min_t2_s"] = 0.001
fit_params["range_t2s_s"] = 0.05

# get the indices of the peaks:
fit_params["metabs"] = metabs
fit_params["fit_range_repetitions"] = repetitions_to_fit
fit_params["range_freqs_Hz"] = 25
fit_params["cut_off"] = 70
fit_params["niter"] = 1 # number of iterations:
fit_params["npoints"] = 21 # number of tested points per iteration:
fit_params["rep_fitting"] = 11 # number of tested points per iteration:
fit_params["provided_dims"] = ["reps", "fid"]
fit_params["use_all_cores"] = True

fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)

# get the indices of the peaks:
peak_indices = ut_spec.find_npeaks(input_data=np.abs(np.sum(np.fft.fftshift(np.fft.fft(np.fft.ifft(np.fft.ifftshift(experiment.complex_spec))[:,fit_params["cut_off"]:])),axis=0)),
                              freq_range=fit_params['freq_range_ppm'],
                              npeaks=len(metabs),
                              find_peaks_params={'distance':150},
                                   plot=True)
# diff literature and measured:

fit_params["metabs_freqs_ppm"] = fit_params["freq_range_ppm"][peak_indices]
    #overwrite the peak values since the hydrate peak is not found for some reason
#fit_params["metabs_freqs_ppm"] = [163.6,171.1,175.9,179.9,183.3]
#fit_params["metabs_freqs_Hz"] = [None, None, None,None]


fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)
fit_params_31=fit_params

fit_spectrums_31, fit_amps_31, fit_freqs_31, fit_t2s_31, fit_stds_31  = ut_fitting.fit_data_pseudo_inv(input_data=experiment.complex_spec,
                                                                             data_obj=experiment,
                                                                  fit_params=fit_params_31,
                                                              use_multiprocessing=True)

fit_freqs_ppm_31 = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs_31), hz_axis=fit_params_31["freq_range_Hz"], ppm_axis=fit_params_31["freq_range_ppm"], ppm_axis_flipped=False)


In [ ]:
## frequency errors
fit_freqs_diff_stds_31 = np.sqrt(np.abs(fit_stds_31[0,0,0,0,:,0,0,1]**2) +
                                        np.abs(fit_stds_31[0,0,0,0,:,0,1,1]**2))

In [ ]:
plot_fit_results(fit_spectrums_31,fit_freqs_31,sig_31,nspect_31deg,fit_params_31)

## Save fit results

In [ ]:
fit_results = {}
fit_results["fit_spectrums_38"] = fit_spectrums_38
fit_results["fit_spectrums_31"] = fit_spectrums_31
fit_results["fit_freqs_38"] = fit_freqs_38
fit_results["fit_freqs_31"] = fit_freqs_31
fit_results["fit_amps_38"] = fit_amps_38
fit_results["fit_amps_31"] = fit_amps_31
fit_results["fit_t2s_38"] = fit_t2s_38
fit_results["fit_t2s_31"] = fit_t2s_31

fit_results["fit_params_38"] = fit_params_38
fit_results["fit_params_31"] = fit_params_31

fit_results["fit_stds_31"] = fit_stds_31
fit_results["fit_stds_38"] = fit_stds_38


import pickle
hypermri.utils.utils_general.save_as_pkl(dir_path=savepath,
                filename='fit_spectra',
                file = fit_results,
                file_keys=fit_results.keys(),
                use_timestamp=False)
                

# Analysis

In [ ]:
time_scale_38deg = np.arange(0,nspect_38deg.method['PVM_RepetitionTime']*nspect_38deg.method['PVM_NRepetitions'],nspect_38deg.method['PVM_RepetitionTime'])/1000
time_scale_31deg = np.arange(0,nspect_31deg.method['PVM_RepetitionTime']*nspect_31deg.method['PVM_NRepetitions'],nspect_31deg.method['PVM_RepetitionTime'])/1000

In [ ]:
def aucr(fit_amps,fit_stds):
    
    pyr_amp=np.abs(np.squeeze(fit_amps[:,:,:,:,:,:,0]))
    lac_amp=np.abs(np.squeeze(fit_amps[:,:,:,:,:,:,1]))

    d_pyr_amp = np.squeeze(fit_stds[..., 0, 0])
    d_lac_amp = np.squeeze(fit_stds[..., 1, 0])
    sum_pyr=np.sum(pyr_amp,axis=0)
    sum_lac=np.sum(lac_amp,axis=0)

    d_sum_pyr = np.sqrt(np.sum(d_pyr_amp**2, axis=0)) 
    d_sum_lac = np.sqrt(np.sum(d_lac_amp**2, axis=0)) 

    AUCR=sum_lac/sum_pyr

    d_AUCR = np.abs(AUCR * np.sqrt((d_sum_pyr / sum_pyr)**2 + (d_sum_lac / sum_lac)**2))
    print('AUCR=',np.round(AUCR,2),'±',np.round(d_AUCR,2))
    return AUCR,d_AUCR

In [ ]:
plt.close('all')
fig,ax=plt.subplots(1,figsize=(single_column_width*0.35,3),tight_layout=True)

pyr_peak_38deg = np.max(np.abs(np.squeeze(fit_amps_38)[:,0]))
pyr_peak_31deg = np.max(np.abs(np.squeeze(fit_amps_31)[:,0]))
lac_pyr_38deg = np.sum(np.abs(np.squeeze(fit_amps_38)[:,1]))/np.sum(np.abs(np.squeeze(fit_amps_38)[:,0]))
lac_pyr_31deg = np.sum(np.abs(np.squeeze(fit_amps_31)[:,1]))/np.sum(np.abs(np.squeeze(fit_amps_31)[:,0]))


aucr_38deg,daucr_38deg=aucr(fit_amps_38,fit_stds_38)
aucr_31deg,daucr_31deg=aucr(fit_amps_31,fit_stds_31)

print(100*(1-aucr_31deg/aucr_38deg))

print(100*np.sqrt((daucr_31deg/aucr_38deg)**2+(daucr_38deg*aucr_31deg/aucr_38deg**2)**2))



ax.plot(time_scale_31deg-22,np.abs(fit_amps_31[0,0,0,0,:,0,0])/pyr_peak_31deg,label='31.8$^\circ$C',color='C0')
ax.plot(time_scale_38deg-24,np.abs(fit_amps_38[0,0,0,0,:,0,0])/pyr_peak_38deg,label='37.8$^\circ$C',color='C3')
ax.plot(time_scale_38deg-24,np.abs(fit_amps_38[0,0,0,0,:,0,1])/pyr_peak_38deg,color='C3',linestyle='dashed')
ax.plot(time_scale_31deg-22,np.abs(fit_amps_31[0,0,0,0,:,0,1])/pyr_peak_31deg,color='C0',linestyle='dashed')

ax.set_xlim([-5,80])
ax.set_yticks([0,0.5,1])
ax.set_ylabel('I [a.u.]')
ax.set_xlabel('t [s]')

print('AUCR: 37.8°C/31.8°C = '+str(np.round(lac_pyr_38deg,2))+'/'+str(np.round(lac_pyr_31deg,2)))
ax.legend(ncols=1)



In [ ]:
fig,ax=plt.subplots(1,figsize=(2,2),tight_layout=True)
ax.bar(['31.8$^\circ$C','37.8$^\circ$C'],[np.round(lac_pyr_31deg,2),np.round(lac_pyr_38deg,2)],color=('C0','C3'),width=0.5)
ax.set_ylim([0,1])
ax.set_xlim([-0.5,1.5])
ax.set_yticks([0,0.5,1])
ax.set_ylabel('AUCR')


# Plot temperature

## Find animal rectal temp from temp dataframe

In [ ]:
nspect_38deg_temp = []
seq_start_time = nspect_38deg.acqp['ACQ_time'][1:20]
TR = nspect_38deg.method['PVM_RepetitionTime']/1000
NR = nspect_38deg.method['PVM_NRepetitions']

# convert this to seconds since January 1 1970 for easier comparison since hours, minutes and seconds are not ideal
seq_end_time_seconds = nspect_38deg.acqp['MetaTimeStamp'].timestamp()
seq_duration = NR*TR
seq_start_time_seconds = seq_end_time_seconds-seq_duration
time_axis_38deg = []
for rep_idx,repetition in enumerate(sig_38):
    current_rep_time = seq_start_time_seconds+repetition*TR
    current_time_index = np.argmin(np.abs(temp_df['EpochTime']-current_rep_time))
    nspect_38deg_temp.append(temp_df['Temperature'].iloc[current_time_index])
    time_axis_38deg.append(current_rep_time-seq_start_time_seconds)
fig,ax=plt.subplots(1)
ax.plot(time_axis_38deg,nspect_38deg_temp)
ax.set_ylabel('T [°C]')

In [ ]:
nspect_31deg_temp = []
seq_start_time = nspect_31deg.acqp['ACQ_time'][1:20]
TR = nspect_31deg.method['PVM_RepetitionTime']/1000
NR = nspect_31deg.method['PVM_NRepetitions']

# convert this to seconds since January 1 1970 for easier comparison since hours, minutes and seconds are not ideal
seq_end_time_seconds = nspect_31deg.acqp['MetaTimeStamp'].timestamp()
seq_duration = NR*TR
seq_start_time_seconds = seq_end_time_seconds-seq_duration
time_axis_31deg = []
for rep_idx,repetition in enumerate(sig_31):
    current_rep_time = seq_start_time_seconds+repetition*TR
    current_time_index = np.argmin(np.abs(temp_df['EpochTime']-current_rep_time))
    nspect_31deg_temp.append(temp_df['Temperature'].iloc[current_time_index])
    time_axis_31deg.append(current_rep_time-seq_start_time_seconds)
fig,ax=plt.subplots(1)
ax.plot(time_axis_31deg,nspect_31deg_temp,'-o')
ax.set_ylabel('T [°C]')

In [ ]:
print(np.mean(nspect_31deg_temp)),print(np.mean(nspect_38deg_temp))

In [ ]:
pa_la_frq_38=np.abs(np.squeeze(fit_freqs_ppm_38)[:,0]-np.squeeze(fit_freqs_ppm_38)[:,1])
pa_la_frq_31=np.abs(np.squeeze(fit_freqs_ppm_31)[:,0]-np.squeeze(fit_freqs_ppm_31)[:,1])

pa_la_frq_38=np.where(pa_la_frq_38==0,np.nan,pa_la_frq_38)
pa_la_frq_31=np.where(pa_la_frq_31==0,np.nan,pa_la_frq_31)

temp_38,temp_err_38=temperature_from_frequency(pa_la_frq_38,'5mM')
temp_31,temp_err_31=temperature_from_frequency(pa_la_frq_31,'5mM')

In [ ]:
temp_38_in_range=temp_38[sig_38[0]:sig_38[-1]]
pa_signal_range_38 = np.abs(np.squeeze(fit_amps_38))[sig_38[0]:sig_38[-1],0]
la_signal_range_38 = np.abs(np.squeeze(fit_amps_38))[sig_38[0]:sig_38[-1],1]
temp_31_in_range=temp_31[sig_31[0]:sig_31[-1]]
pa_signal_range_31 = np.abs(np.squeeze(fit_amps_31))[sig_31[0]:sig_31[-1],0]
la_signal_range_31 = np.abs(np.squeeze(fit_amps_31))[sig_31[0]:sig_31[-1],1]

In [ ]:
## temperature errors from fits
df_ppm_38deg = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs_diff_stds_38), data_obj=nspect_38deg,ppm_centered_at_0=True)
df_ppm_31deg = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs_diff_stds_31), data_obj=nspect_31deg,ppm_centered_at_0=True)

dT_38deg = np.abs(temperature_from_frequency(pa_la_frq_38[sig_38[0]:sig_38[-1]]+df_ppm_38deg[sig_38[0]:sig_38[-1]],'5mM')[0]-temperature_from_frequency(pa_la_frq_38[sig_38[0]:sig_38[-1]],'5mM')[0])
dT_31deg = np.abs(temperature_from_frequency(pa_la_frq_31[sig_31[0]:sig_31[-1]]+df_ppm_31deg[sig_31[0]:sig_31[-1]],'5mM')[0]-temperature_from_frequency(pa_la_frq_31[sig_31[0]:sig_31[-1]],'5mM')[0])


In [ ]:
# Time between start of sequence and injection
time_start_to_inj=23

In [ ]:
high_signal_reps_38 = np.squeeze(np.where(dT_38deg<2))
high_signal_reps_31 = np.squeeze(np.where(dT_31deg<2))

In [ ]:
plt.close('all')
fig,ax=plt.subplots(1,figsize=(single_column_width*0.5,3),tight_layout=True)
ax.errorbar(time_scale_38deg[sig_38[0]:sig_38[-1]][high_signal_reps_38]-time_start_to_inj,temperature_from_frequency(pa_la_frq_38,'5mM')[0][sig_38[0]:sig_38[-1]][high_signal_reps_38],capsize=2,
            label=r'$T_{C13,hot}$',fmt='-o',yerr=dT_38deg[high_signal_reps_38],color='C3',markersize=3)
ax.errorbar(time_scale_31deg[sig_31[0]:sig_31[-1]][high_signal_reps_31]-time_start_to_inj,temperature_from_frequency(pa_la_frq_31,'5mM')[0][sig_31[0]:sig_31[-1]][high_signal_reps_31],capsize=2,
            yerr=dT_31deg[high_signal_reps_31],label=r'$T_{C13,cold}$',fmt='-o',color='C0',markersize=3)

ax.set_yticks([27,32,37])

ax.set_xlabel('t[s]')
ax.set_ylabel('T [$^\circ$C]')
ax.errorbar(np.array(time_axis_38deg)[high_signal_reps_38]-time_start_to_inj,np.array(nspect_38deg_temp)[high_signal_reps_38],fmt='-o',label='$T_{rec}$',color='gray',yerr=[0.1],capsize=2,markersize=3)
ax.errorbar(np.array(time_axis_31deg)[high_signal_reps_31]-time_start_to_inj,np.array(nspect_31deg_temp)[high_signal_reps_31],fmt='-o',color='gray',yerr=[0.1],capsize=2,markersize=3)


print('High rectal temp:',np.mean(np.array(nspect_38deg_temp)[high_signal_reps_38]))
print('Low rectal temp:',np.mean(np.array(nspect_31deg_temp)[high_signal_reps_31]))


ax.legend()



In [ ]:
print('Mean Temp hot',np.nanmean(temperature_from_frequency(pa_la_frq_38,'5mM')[0][sig_38[0]:sig_38[-1]][high_signal_reps_38]))
print('Std Temp hot',np.nanstd(temperature_from_frequency(pa_la_frq_38,'5mM')[0][sig_38[0]:sig_38[-1]][high_signal_reps_38]))


In [ ]:
print('Mean Temp cold',np.nanmean(temperature_from_frequency(pa_la_frq_31,'5mM')[0][sig_31[0]:sig_31[-1]][high_signal_reps_31]))
print('Std Temp cold',np.nanstd(temperature_from_frequency(pa_la_frq_31,'5mM')[0][sig_31[0]:sig_31[-1]][high_signal_reps_31]))


# Plotting timecurves against each other

In [ ]:
plt.close('all')
fig,ax=plt.subplots(1,figsize=(3,3),tight_layout=True)
pyr_peak_38deg = np.max(peak_integrals_38deg[:,1])
pyr_peak_31deg = np.max(peak_integrals_31deg[:,1])

ax.plot(time_scale_31deg-20,peak_integrals_31deg[:,1]/pyr_peak_38deg,label='31.8°C',color='C0')
ax.plot(time_scale_38deg-20,peak_integrals_38deg[:,1]/pyr_peak_38deg,label='37.9°C',color='C3')
ax.plot(time_scale_38deg-20,peak_integrals_38deg[:,0]/pyr_peak_38deg,color='C3',linestyle='dashed')
ax.plot(time_scale_31deg-20,peak_integrals_31deg[:,0]/pyr_peak_38deg,color='C0',linestyle='dashed')

ax.set_xlim([-5,80])
ax.set_ylabel('I [a.u.]')
ax.set_xlabel('t [s]')
ax.set_yticks([0,0.5,1])
print('AUCR: 37.9°C/31.8°C = '+str(np.round(lac_pyr_38deg,2))+'/'+str(np.round(lac_pyr_31deg,2)))
ax.legend(ncols=1)


In [ ]:
fig,ax=plt.subplots(1,figsize=(2,2),tight_layout=True)
ax.bar(['31.8°C','37.9°C'],[0.39,0.72],color=('C0','C3'),width=0.5)
ax.set_ylim([0,1])
ax.set_xlim([-0.5,1.5])
ax.set_yticks([0,0.5,1])
ax.set_ylabel('AUCR')


# Plotting exemplary spectra

In [ ]:

plt.close('all')
fig,ax=plt.subplots(1,figsize=(5,3),tight_layout=True)
freq_ax_38 = ut_spec.get_freq_axis(nspect_38deg)
freq_ax_31 = ut_spec.get_freq_axis(nspect_31deg)

ax.plot(freq_ax_38,nspect_38deg.spec[18,:]/np.max(nspect_38deg.spec[18,:],axis=0),label='37.9°C',color='C3')
ax.plot(freq_ax_31,nspect_31deg.spec[21,:]/np.max(nspect_38deg.spec[21,:],axis=0),label='31.8°C',color='C0')
ax.legend()
ax.set_xlabel('ppm')
ax.set_ylabel('I [a.u.]')
ax.set_xlim([186,169])
ax.set_xticks([185,180,175,170])

fig,ax=plt.subplots(1,figsize=(2,2),tight_layout=True)
freq_ax_38 = ut_spec.get_freq_axis(nspect_38deg)
freq_ax_31 = ut_spec.get_freq_axis(nspect_31deg)

ax.plot(freq_ax_38,nspect_38deg.spec[18,:]/np.max(nspect_38deg.spec[18,:],axis=0),label='37.9°C',color='C3')
ax.plot(freq_ax_31,nspect_31deg.spec[21,:]/np.max(nspect_38deg.spec[21,:],axis=0),label='31.8°C',color='C0')
ax.set_xlim([172,170])


fig,ax=plt.subplots(1,figsize=(2,2),tight_layout=True)
freq_ax_38 = ut_spec.get_freq_axis(nspect_38deg)
freq_ax_31 = ut_spec.get_freq_axis(nspect_31deg)

ax.plot(freq_ax_38,nspect_38deg.spec[18,:]/np.max(nspect_38deg.spec[18,:],axis=0),label='37.9°C',color='C3')
ax.plot(freq_ax_31,nspect_31deg.spec[21,:]/np.max(nspect_38deg.spec[21,:],axis=0),label='31.8°C',color='C0')
ax.set_xlim([184,182])
ax.set_ylim([0.1,0.65])


# Plot anatomical with NSPECT slice

In [ ]:
from hypermri.utils.plot import plot_NSPECT_slice_orientation_in_anatomical_image

In [ ]:
plot_NSPECT_slice_orientation_in_anatomical_image.plot_image_stack(nspect_38deg,coronal,'',line_color='w',plotname='_31deg',saveplot=True,plotsavepath=savepath,file_format='svg',initial_slice_number=19,vmin=2,vmax=60)